# Setup

In [104]:
!pip install -U llama-index > /dev/null 2>&1

!pip install llama-index-readers-file llama-index-embeddings-huggingface llama-index-llms-ollama > /dev/null 2>&1

!sudo apt-get install -y pciutils > /dev/null 2>&1

!curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1 # download ollama api

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid de

In [2]:
from IPython.display import clear_output

In [103]:
import os

import threading

import subprocess

import requests

import json

from llama_index.llms.ollama import Ollama

from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import SummaryIndex, VectorStoreIndex

from llama_index.core.tools import QueryEngineTool

from llama_index.core.node_parser import SentenceSplitter

from llama_index.core.vector_stores import MetadataFilters

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

from llama_index.core.vector_stores import FilterCondition

from llama_index.core.agent import FunctionCallingAgentWorker

from llama_index.core.agent import AgentRunner

from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.objects import ObjectIndex

from llama_index.core.query_engine.router_query_engine import RouterQueryEngine

from llama_index.core.selectors import LLMSingleSelector

from llama_index.core.postprocessor import SentenceTransformerRerank

from llama_index.core.schema import TextNode, NodeWithScore

from llama_index.core.schema import QueryBundle

from llama_index.core.evaluation import (
    BatchEvalRunner,
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    RetrieverEvaluator
)

from copy import deepcopy

from typing import List

import nest_asyncio

import pandas as pd

import asyncio



nest_asyncio.apply()

In [5]:
def ollama():

    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'

    os.environ['OLLAMA_ORIGINS'] = '*'

    subprocess.Popen(["ollama", "serve"])

In [6]:
ollama_thread = threading.Thread(target=ollama)

ollama_thread.start()

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIHFMqNLP2h+iCTdOrjPUmdKSWppoxhU5+zFTVmxG9sOy



2024/11/05 05:32:31 routes.go:1158: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-11-05T05:32:31.148Z level=INFO source=images.go:754 msg="total blobs: 

In [7]:
!ollama pull llama3.2 > /dev/null 2>&1

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
time=2024-11-05T05:32:47.696Z level=INFO source=common.go:49 msg="Dynamic LLM libraries" runners="[cpu cpu_avx cpu_avx2 cuda_v11 cuda_v12 rocm_v60102]"
time=2024-11-05T05:32:47.696Z level=INFO source=gpu.go:221 msg="looking for compatible GPUs"


[GIN] 2024/11/05 - 05:32:48 | 200 |      68.409µs |       127.0.0.1 | HEAD     "/"


time=2024-11-05T05:32:48.011Z level=INFO source=types.go:123 msg="inference compute" id=GPU-e2980c27-8e6f-7002-c8e7-21d660d3e883 library=cuda variant=v12 compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-11-05T05:32:48.011Z level=INFO source=types.go:123 msg="inference compute" id=GPU-f565c170-d708-25e3-62a4-cca0eeba9f55 library=cuda variant=v12 compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-11-05T05:32:49.034Z level=INFO source=download.go:175 msg="downloading dde5aa3fc5ff in 16 126 MB part(s)"
time=2024-11-05T05:32:55.425Z level=INFO source=download.go:175 msg="downloading 966de95ca8a6 in 1 1.4 KB part(s)"
time=2024-11-05T05:32:57.283Z level=INFO source=download.go:175 msg="downloading fcc5a6bec9da in 1 7.7 KB part(s)"
time=2024-11-05T05:32:58.967Z level=INFO source=download.go:175 msg="downloading a70ff7e570d9 in 1 6.0 KB part(s)"
time=2024-11-05T05:33:00.712Z level=INFO source=download.go:175 msg="downlo

[GIN] 2024/11/05 - 05:33:10 | 200 | 22.470198465s |       127.0.0.1 | POST     "/api/pull"


In [8]:
llm = Ollama(model="llama3.2", request_timeout=120.0)

In [9]:
llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c67ba1a6d40>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7c68c941fd90>, completion_to_prompt=<function default_completion_to_prompt at 0x7c68c9384280>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.2', temperature=0.75, context_window=3900, request_timeout=120.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None)

In [11]:
embed_model = HuggingFaceEmbedding(

    model_name="BAAI/bge-small-en-v1.5"

)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
Settings.llm = llm

Settings.embed_model = embed_model

In [10]:
# load documents

typology_documents = SimpleDirectoryReader(input_files=["/kaggle/input/linguistics-books/typology.pdf"]).load_data()

morphology_documents = SimpleDirectoryReader(input_files=["/kaggle/input/linguistics-books/morphology.pdf"]).load_data()

phonetics_documents = SimpleDirectoryReader(input_files=["/kaggle/input/linguistics-books/phonetics.pdf"]).load_data()

# Baseline

In [13]:
splitter = SentenceSplitter(chunk_size=1024)

typology_nodes = splitter.get_nodes_from_documents(typology_documents)

morphology_nodes = splitter.get_nodes_from_documents(morphology_documents)

phonetics_nodes = splitter.get_nodes_from_documents(phonetics_documents)

In [14]:
#summary_index = SummaryIndex(nodes)

typology_vector_index = VectorStoreIndex(typology_nodes)

morphology_vector_index = VectorStoreIndex(morphology_nodes)

phonetics_vector_index = VectorStoreIndex(phonetics_nodes)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
"""summary_query_engine = summary_index.as_query_engine(

    response_mode="tree_summarize",

    use_async=True

)"""

typology_query_engine = typology_vector_index.as_query_engine(similarity_top_k=3)

morphology_query_engine = morphology_vector_index.as_query_engine(similarity_top_k=3)

phonetics_query_engine = phonetics_vector_index.as_query_engine(similarity_top_k=3)

#vector_query_engine = vector_index.as_query_engine()

In [16]:
typology_tool = QueryEngineTool.from_defaults(

    name="typology_tool",

    query_engine=typology_query_engine,

    description=(

        "Useful for retrieving specific context about typology, which studies and classifies languages according to their structural features to allow their comparison. Its aim is to describe and explain the structural diversity and the common properties of the world's languages."

    ),

)

morphology_tool = QueryEngineTool.from_defaults(

    name="morphology_tool",

    query_engine=morphology_query_engine,

    description=(

        "Useful for retrieving specific context about morphology, which is the study of words, including the principles by which they are formed, and how they relate to one another within a language."

    ),

)

phonetics_tool = QueryEngineTool.from_defaults(

    name="phonetics_tool",

    query_engine=phonetics_query_engine,

    description=(

        "Useful for retrieving specific context about phonetics and phonology, meaning the speech sounds that constitute the fundamental components of a language, and the relationships between these sounds."

    ),

)

In [17]:
baseline_query_engine = RouterQueryEngine(

    selector=LLMSingleSelector.from_defaults(),

    query_engine_tools=[

        typology_tool,

        morphology_tool,

        phonetics_tool

    ],

    verbose=True

)

In [19]:
response = baseline_query_engine.query("Explain the basics of how words are formed in Semitic Languages.")

print(str(response))

[GIN] 2024/11/05 - 05:36:38 | 200 |  1.107375008s |       127.0.0.1 | POST     "/api/chat"
Selecting query engine 1: Useful for retrieving specific context about morphology, which is the study of words, including the principles by which they are formed, and how they relate to one another within a language..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[GIN] 2024/11/05 - 05:36:45 | 200 |  7.408307354s |       127.0.0.1 | POST     "/api/chat"
In Semitic languages, such as Arabic and Hebrew, word formation is often derived from a combination of linguistic processes including derivation and compounding. 

One key aspect of word formation in these languages is the role of suffixes and prefixes. Suffixes are added to roots to indicate grammatical function or to form new words, while prefixes can modify the root's meaning. For example, in Arabic, the root "k-t-b" (to write) can be modified by adding a prefix "q-" to form the verb "q-taba" (to record). 

Another important aspect of word formation in Semitic languages is the use of roots to convey meaning. Each root typically has multiple forms and variations, which are used to express different meanings or grammatical functions. For instance, the root "s-l-m" (to be at peace) can take on various forms such as "salam", "salama", or "aslama", each with its own connotation or grammatical funct

# Baseline Evaluation

In [54]:
eval_questions = ["tell me about the subfamiles of indo-european languages",
                  "describe the basics of morphology in Semitic languages such as Arabic",
                  "what is a typological characteristic of Sino-Tibetan languages such as Chinese?",
                  "describe the phonetics of slavic languages"]

In [52]:
correctness_evaluator = CorrectnessEvaluator(llm = llm)
faithfulness_evaluator = FaithfulnessEvaluator(llm = llm)
relevancy_evaluator = RelevancyEvaluator(llm = llm)

In [78]:
async def evaluate_async(query_eng):
    # Initialize the BatchEvalRunner
    runner = BatchEvalRunner(
        {
            "correctness": correctness_evaluator,
            "faithfulness": faithfulness_evaluator,
            "relevancy": relevancy_evaluator
        },
        show_progress = False
    )

    # Run the asynchronous evaluation
    eval_result = await runner.aevaluate_queries(
        query_engine = query_eng,
        queries = eval_questions,
        #queries = [question.query for question in eval_questions]
    )

    return eval_result

In [79]:
def eval_res(eval_questions,result):
    data = []
    for i, question in enumerate(eval_questions):
        correctness_result = result['correctness'][i]
        faithfulness_result = result['faithfulness'][i]
        relevancy_result = result['relevancy'][i]
        data.append({
            'Query': question,
            'Correctness response': correctness_result.response,
            'Correctness passing': correctness_result.passing,
            'Correctness feedback': correctness_result.feedback,
            'Correctness score': correctness_result.score,
            'Faithfulness response': faithfulness_result.response,
            'Faithfulness passing': faithfulness_result.passing,
            'Faithfulness feedback': faithfulness_result.feedback,
            'Faithfulness score': faithfulness_result.score,
            'Relevancy response': relevancy_result.response,
            'Relevancy passing': relevancy_result.passing,
            'Relevancy feedback': relevancy_result.feedback,
            'Relevancy score': relevancy_result.score,
        })
    return pd.DataFrame(data)

In [116]:
result = asyncio.run(evaluate_async(baseline_query_engine))

[GIN] 2024/11/05 - 07:37:48 | 200 |  1.285286403s |       127.0.0.1 | POST     "/api/chat"
Selecting query engine 1: Morphology studies the formation of words, which would include the subfamilies of Indo-European languages..
[GIN] 2024/11/05 - 07:37:49 | 200 |  1.472543509s |       127.0.0.1 | POST     "/api/chat"
Selecting query engine 1: Useful for retrieving specific context about morphology, which is the study of words, including the principles by which they are formed, and how they relate to one another within a language..
[GIN] 2024/11/05 - 07:38:03 | 200 | 14.144651664s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:38:03 | 200 | 14.909027771s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:38:04 | 200 |  1.633220626s |       127.0.0.1 | POST     "/api/chat"
Selecting query engine 0: Typological characteristic of Sino-Tibetan languages, such as Chinese, refers to the study and classification of structural features that allow comparison across these

In [117]:
df = eval_res(eval_questions,result)

In [118]:
df

,Query,Correctness response,Correctness passing,Correctness feedback,Correctness score,Faithfulness response,Faithfulness passing,Faithfulness feedback,Faithfulness score,Relevancy response,Relevancy passing,Relevancy feedback,Relevancy score
0,tell me about the subfamiles of indo-european ...,The Indo-European language family is further d...,True,The generated answer is fully relevant to the ...,4.5,The Indo-European language family is further d...,False,"Based on the provided context, I would answer:...",0.0,The Indo-European language family is further d...,False,"NO, the response for the query ""tell me about ...",0.0
1,describe the basics of morphology in Semitic l...,"In Semitic languages like Arabic, morphology i...",True,The generated answer provides a comprehensive ...,4.0,"In Semitic languages like Arabic, morphology i...",True,Information: Morphology is the study of the in...,1.0,"In Semitic languages like Arabic, morphology i...",False,NO,0.0
2,what is a typological characteristic of Sino-T...,One common feature among Sino-Tibetan language...,True,The generated answer is relevant as it address...,4.0,One common feature among Sino-Tibetan language...,True,"Based on the context provided, I will answer a...",1.0,One common feature among Sino-Tibetan language...,True,"YES. The response for the query ""what is a typ...",1.0
3,describe the phonetics of slavic languages,The study of Slavic languages reveals a divers...,True,The generated answer addresses all aspects of ...,4.0,The study of Slavic languages reveals a divers...,False,NO \n\nInformation: The study of Slavic langua...,0.0,The study of Slavic languages reveals a divers...,False,NO. The response does not describe the phoneti...,0.0


In [119]:
print("Evaluation: Baseline\n")
corr = df["Correctness score"].mean()
faith = df["Faithfulness score"].mean()
rel = df["Relevancy score"].mean()
print(f"Correctness: {corr/5}")
print(f"Faithfulness: {faith}")
print(f"Relevancy: {rel}")

Evaluation: Baseline

Correctness: 0.825
Faithfulness: 0.5
Relevancy: 0.25


# 2nd Approach: Agentic RAG with Sentence-Window Retrieval

In [105]:
def build_tool(
    name,
    description,
    documents,
    llm=llm,
    embed_model=embed_model,
    sentence_window_size=8,
    similarity_top_k=6,
    rerank_top_n=3
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    nodes = splitter.get_nodes_from_documents(documents)

    vector_index = VectorStoreIndex(nodes)

    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = vector_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )

    tool = QueryEngineTool.from_defaults(name=name,
                                         query_engine=typology_query_engine,
                                         description=description,
                                         )
    return tool

In [106]:
typology_tool = build_tool(name="typology_tool",
                           documents=typology_documents,
                           description="Useful for retrieving specific context about typology, which studies and classifies languages according to their structural features to allow their comparison. Its aim is to describe and explain the structural diversity and the common properties of the world's languages.")
morphology_tool = build_tool(name="morphology_tool",
                             documents=morphology_documents,
                             description="Useful for retrieving specific context about morphology, which is the study of words, including the principles by which they are formed, and how they relate to one another within a language.")
phonetics_tool = build_tool(name="phonetics tool",
                            documents=phonetics_documents,
                            description="Useful for retrieving specific context about phonetics and phonology, meaning the speech sounds that constitute the fundamental components of a language, and the relationships between these sounds.")

In [107]:
obj_index = ObjectIndex.from_objects(

    [typology_tool,

     morphology_tool,

     phonetics_tool

     ],

    index_cls=VectorStoreIndex,

)

obj_retriever = obj_index.as_retriever(similarity_top_k=2)

In [114]:
agent_worker = FunctionCallingAgentWorker.from_tools(tool_retriever=obj_retriever,llm=llm,verbose=True)
agent = AgentRunner(agent_worker)

In [115]:
response = agent.query(

    "Tell me briefly about the basics and main points of morphology in Semitic languages such as Arabic, "

    "then tell me about the main points of their phonetics, "

    "and then tell me how it sets them apart from other kinds of languages in terms of their typology."

)

Added user message to memory: Tell me briefly about the basics and main points of morphology in Semitic languages such as Arabic, then tell me about the main points of their phonetics, and then tell me how it sets them apart from other kinds of languages in terms of their typology.
[GIN] 2024/11/05 - 07:36:46 | 200 |  3.331473645s |       127.0.0.1 | POST     "/api/chat"
=== Calling Function ===
Calling function: morphology_tool with args: {"input": "The basics and main points of morphology in Semitic languages such as Arabic include root patterns, trilateral roots, pattern conjugation, pattern declension, and patterns for verb conjugations."}
[GIN] 2024/11/05 - 07:36:50 | 200 |  3.881937353s |       127.0.0.1 | POST     "/api/chat"
=== Function Output ===
In Semitic languages like Arabic, morphology is characterized by a three-consonant root system. This root system serves as the foundation for words, which are derived from it through various affixes that indicate grammatical function

# 2nd Approach Evaluation

In [110]:
agent_worker = FunctionCallingAgentWorker.from_tools(tool_retriever=obj_retriever,llm=llm,verbose=False)
agent = AgentRunner(agent_worker)

In [111]:
result = asyncio.run(evaluate_async(agent))
df = eval_res(eval_questions,result)
df

[GIN] 2024/11/05 - 07:35:56 | 200 |  758.564483ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:35:56 | 200 |  791.111931ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:35:58 | 200 |  1.713415369s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:00 | 200 |  2.519583866s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:01 | 200 |  4.730034222s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:01 | 200 |  781.603884ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:03 | 200 |  1.411867839s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:07 | 200 |  5.556281277s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:07 | 200 |  3.879241426s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:07 | 200 |  512.678798ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/11/05 - 07:36:08 | 200 |  761.028512ms |       127.0.0.1 | POST     "/api/chat"

,Query,Correctness response,Correctness passing,Correctness feedback,Correctness score,Faithfulness response,Faithfulness passing,Faithfulness feedback,Faithfulness score,Relevancy response,Relevancy passing,Relevancy feedback,Relevancy score
0,tell me about the subfamiles of indo-european ...,<|python_tag|>from langtools import langcodes\...,True,The generated answer is relevant to the user q...,4.0,<|python_tag|>from langtools import langcodes\...,False,NO\nThe context does not mention anything rela...,0.0,<|python_tag|>from langtools import langcodes\...,False,NO. The response does not provide any informat...,0.0
1,describe the basics of morphology in Semitic l...,"In Arabic, there are several types of roots th...",True,The generated answer provides a comprehensive ...,4.0,"In Arabic, there are several types of roots th...",False,NO \n\nThe provided context does not support t...,0.0,"In Arabic, there are several types of roots th...",False,NO. The response provided does not mention the...,0.0
2,what is a typological characteristic of Sino-T...,This is a typological characteristic of many S...,True,The generated answer provides a clear and rele...,4.0,This is a typological characteristic of many S...,True,"For the first information, I would answer YES....",1.0,This is a typological characteristic of many S...,False,"NO. The response for the query ""what is a typo...",0.0
3,describe the phonetics of slavic languages,The phonetics of Slavic languages can be descr...,True,The generated answer provides a detailed and c...,4.5,The phonetics of Slavic languages can be descr...,False,NO \n\nAlthough the text does describe the pho...,0.0,The phonetics of Slavic languages can be descr...,False,NO. The response does not match the context in...,0.0


In [113]:
print("Evaluation: Baseline\n")
corr = df["Correctness score"].mean()
faith = df["Faithfulness score"].mean()
rel = df["Relevancy score"].mean()
print(f"Correctness: {corr/5}")
print(f"Faithfulness: {faith}")
print(f"Relevancy: {rel}")

Evaluation: Baseline

Correctness: 0.825
Faithfulness: 0.25
Relevancy: 0.0


# TO DO:
## - Use TruLens for evaluation
## - Deploy on a local API